In [136]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum

In [137]:
def uFunction(k,i): #Working Day probability
    if k in [6, 7, 13, 14]:
        return 0
    else:
        return flatP(t= k-i)

def failureProd(k,i):
    product = 1
    for l in range(i+1, k):
        product = product*(1-uFunction(l, i))
    return product

def constant(r, i):
    total = 0
    for k in range(r, i+8):
        total += uFunction(k,i)*failureProd(k,i)
    return total
        
    
def flatP(t, p0 = 0.318, changeDay = 3):
    if t <= changeDay:
        return p0
    else:
        return (14-t)*p0/(14-changeDay)
    
def alpha_t(a0, a_change, changeDay, t):
    beta_2 = (a0 - a_change)/changeDay
    if t <= changeDay:
        return a0 - beta_2 * t
    else:
        return a0*(14-t)/(14-changeDay) - beta_2 *changeDay* (14 - t)/(14- changeDay) + (t-changeDay)/(14-changeDay)
    
def product(i,j,x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind', 1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    prod = 1
    for k in range (i+1, j):
        prod = prod*(1-tau[k]*flatP(p0, k-i, changeDay))*(x[k]* (alpha_t(a0, a_change, changeDay, j-i) - 1) + 1)
    return prod

def test_product(i,j,x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind',1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    total = 0
    prod = 1
    for t in range(j, i+8): 
        prod = 1
        for k in range(j, t-1):
            prod = prod * (1-tau[k]*flatP(p0, k-i, changeDay))
        prod = prod*tau[t]*flatP(p0, t, changeDay)
        total = total + prod
    return total

def objective(i, x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    tau = ['ind',1,1,1,1,1,0,0,1,1,1,1,1,0,0]
    #print(x[i+1])
    total = (x[i+1]*(alpha_t(a0, a_change, changeDay, 1) - 1) + 1)*tau[i+1]*flatP(p0, 1, changeDay)
    for j in range(i+2, i+8):
        total = total + tau[j]*flatP(p0, j-i, changeDay)*(x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1) + 1)*(product(i,j,x))
    return total

def twoTests(i, x, changeDay = 3, a0=0.4, a_change=0.3, p0=0.318):
    total = 0
    for j in range(i+1, i+8):
        total = total + x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*test_product(i,j,x)
    for m in range(i+1, i+8):
        for n in range(m+1, i+8):
            #total = total + u[m,n]*(alpha_t(a0, a_change, changeDay, m-i) - 1)*(alpha_t(a0, a_change, changeDay, n-i) - 1)*(product(i,n,x))
            total = total + u[m,n]*(alpha_t(a0, a_change, changeDay, m-i) - 1)*(alpha_t(a0, a_change, changeDay, n-i) - 1)*test_product(i,j,x)
    return total

In [82]:
# Create a new model
m = gp.Model("MIP-test")
i=1
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(i+15, i+15, vtype=GRB.BINARY)
#Create a list of variable x[0] to x[14]
print(x)
risk = twoTests(i, x)
# Set objective:
m.setObjective(risk, GRB.MINIMIZE)
#m.setObjective(-10*x[1] - 5*x[2] - 3*x[1]*u, GRB.MINIMIZE)
# Add constraints
m.addConstr(x[1]+ x[2] + x[3] + x[4]+ x[5] + x[6] + x[7]+ x[8] + x[9] + x[10]+ x[11] + x[12] + x[13] + x[14]== 1 , "c0")
for k in range(i+1, i+8):
    for y in range(k+1, i+8):
        m.addConstr(x[k]*x[y] == u[k,y])

m.optimize()

mylist = []
for v in x.values():
    #print(v.x)
    mylist.append(v.x)
print(mylist)
print(objective(i,mylist))

{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>, 4: <gurobi.Var *Awaiting Model Update*>, 5: <gurobi.Var *Awaiting Model Update*>, 6: <gurobi.Var *Awaiting Model Update*>, 7: <gurobi.Var *Awaiting Model Update*>, 8: <gurobi.Var *Awaiting Model Update*>, 9: <gurobi.Var *Awaiting Model Update*>, 10: <gurobi.Var *Awaiting Model Update*>, 11: <gurobi.Var *Awaiting Model Update*>, 12: <gurobi.Var *Awaiting Model Update*>, 13: <gurobi.Var *Awaiting Model Update*>, 14: <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1 rows, 271 columns and 14 nonzeros
Model fingerprint: 0x3013a02b
Model has 21 quadratic constraints
Variable types: 0 continuous, 271 integer (271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix

In [83]:
#two tests candidate
solSet = []
for t1 in range (0, 2):
    for t2 in range (0,2):
        for t3 in range (0,2):
            for t4 in range (0,2):
                for t5 in range (0,2):
                    for t6 in range(0,2):
                        for t7 in range(0,2):
                            if t1+t2+t3+t4+t5+t6+t7 == 2:
                                solSet.append([t1,t2,t3,t4,t5,t6,t7])
print(solSet)

[[0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0]]


In [108]:
result = []
for i in range(1, 8):
    row = []
    for sol in solSet:
        initSol = ['nal', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        for k in range (7):
            initSol[i+k+1] = sol[k]
        string = []
        for k in range (1, 15):
            if initSol[k] == 1:
                if k >= 8:
                    string.append(k-7)
                else:
                    string.append(k)
        string.sort()
        val = objective(i, initSol)
        row.append([string, val])
        row.sort()
    valList = []
    print(row)
    for item in row:
        valList.append(item[1])
    result.append(valList)
df = pd.DataFrame(result)
df = df.transpose()
df.to_csv('twotests.csv')

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
[[[1, 2], 0.28064544081684745], [[1, 3], 0.4820454408168474], [[1, 4], 0.6266294408168475], [[1, 5], 0.7301660432168474], [[1, 6], 0.7885230372968474], [[1, 7], 0.7885230372968474], [[2, 3], 0.18016906831018076], [[2, 4], 0.22836373497684742], [[2, 5], 0.25942471569684744], [[2, 6], 0.28064544081684745], [[2, 7], 0.28064544081684745], [[3, 4], 0.4297637349768474], [[3, 5], 0.4608247156968474], [[3, 6], 0.4820454408168474], [[3, 7], 0.4820454408168474], [[4, 5], 0.6054087156968474], [[4, 6], 0.6266294408168475], [[4, 7], 0.6266294408168475], [[5, 6], 0.7301660432168474], [[5, 7], 0.7301660432168474], [[6, 7], 0.7885230372968474]]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
[[[1, 2], 0.7339751559348539], [[1, 3], 0.266119847031174], [[1, 4], 0.46751984703117394], [[1, 5], 0.6121038470311739], [[1, 6], 0.7156404494311739], [[1, 7], 0.7156404494311739], [[2, 3], 0.28445455353485394], [[2, 4], 0.4858545535348539], [[2, 5], 0.6304385535348539], [[2, 6]

In [87]:
MonTue = []
MonWed = []
MonThu = []
MonFri = []
MonSat = []
MonSun = []
TueWed = []
TueThu = []
TueFri = []
TueSat = []
TueSun = []
WedThu = []
WedFri = []
WedSat = []
WedSun = []
ThuFri = []
ThuSat = []
ThuSun = []
FriSat = []
FriSun = []
SatSun = []
m1 = 1
m2 = 2
if (m1 == 1 or m1 == 8):
    if (m2 == 2 or m2 ==)
    print('hi')

hi


In [90]:
#Reformulate TwoTest Optimization Problem
m = gp.Model("MIP-test")
i=1
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(i+15, i+15, vtype=GRB.BINARY)
#Create a list of variable x[0] to x[14]
print(x)
risk = twoTests(i, x)
# Set objective:
m.setObjective(risk, GRB.MINIMIZE)
#m.setObjective(-10*x[1] - 5*x[2] - 3*x[1]*u, GRB.MINIMIZE)
# Add constraints
m.addConstr(x[1]+ x[2] + x[3] + x[4]+ x[5] + x[6] + x[7]+ x[8] + x[9] + x[10]+ x[11] + x[12] + x[13] + x[14]== 1 , "c0")
for k in range(i+1, i+8):
    for y in range(k+1, i+8):
        m.addConstr(x[k]*x[y] == u[k,y])

m.optimize()

mylist = []
for v in x.values():
    #print(v.x)
    mylist.append(v.x)
print(mylist)
print(objective(i,mylist))

'mkmkmk'

In [115]:
m = gp.Model("MIP-test")
x = m.addVars(15, vtype=GRB.BINARY)
u = m.addVars(15, 15, vtype=GRB.BINARY)

TypeError: 'tupledict' object is not callable

In [119]:
#Create 2D decision list
twoDecisionList = []
for i in range (2, 9):
    for j in range (i+1, i+7):
        twoDecisionList.append([i,j])
for i in range (9, 14):
    for j in range (i+1,15):
        twoDecisionList.append([i,j])
twoDecisionList

[[2, 3],
 [2, 4],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [4, 5],
 [4, 6],
 [4, 7],
 [4, 8],
 [4, 9],
 [4, 10],
 [5, 6],
 [5, 7],
 [5, 8],
 [5, 9],
 [5, 10],
 [5, 11],
 [6, 7],
 [6, 8],
 [6, 9],
 [6, 10],
 [6, 11],
 [6, 12],
 [7, 8],
 [7, 9],
 [7, 10],
 [7, 11],
 [7, 12],
 [7, 13],
 [8, 9],
 [8, 10],
 [8, 11],
 [8, 12],
 [8, 13],
 [8, 14],
 [9, 10],
 [9, 11],
 [9, 12],
 [9, 13],
 [9, 14],
 [10, 11],
 [10, 12],
 [10, 13],
 [10, 14],
 [11, 12],
 [11, 13],
 [11, 14],
 [12, 13],
 [12, 14],
 [13, 14]]

In [140]:
#Initialize
changeDay = 3
a0=0.4
a_change=0.3
p0=0.318

#Gurobi optimization 
m = gp.Model("MIP-test")
x = m.addVars(15, vtype=GRB.BINARY)
y = m.addVars(15, 15, vtype=GRB.BINARY)
z = m.addVar(vtype=GRB.CONTINUOUS, name="z")
#Create a list of variable x[0] to x[14]
#Create a list of variable y[0,0] to y[14,14]
for decision in twoDecisionList:
    j = decision[0]
    r = decision[1]
    m.addConstr(y[j,r] <= x[j], 'c%s 1'%(str(decision)))
    m.addConstr(y[j,r] <= x[r], 'c%s 2'%(str(decision)))
    m.addConstr(y[j,r] >= x[r] + x[j] - 1, 'c%s 3'%(str(decision)))

for i in range(1, 8):
    m.addConstr(z >= x[i+1]*(alpha_t(a0, a_change, changeDay, 1) - 1)*uFunction(i+1, i) + uFunction(i+1, i) + quicksum(x[j]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*constant(j,i) for j in range(i+1, i+8)) + quicksum(y[j,r]*(alpha_t(a0, a_change, changeDay, j-i) - 1)*(alpha_t(a0, a_change, changeDay, r-i) - 1)*constant(r,i) for j in range(i+1, i+7) for r in range(j+1, i+8)) + constant(i+2, i), "myconstraint")

m.addConstr(quicksum(x[k] for k in range(15)) == 2)
m.setObjective(z, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 179 rows, 241 columns and 597 nonzeros
Model fingerprint: 0xc2f7deac
Variable types: 1 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 2e+00]
Found heuristic solution: objective 0.8331630
Presolve removed 0 rows and 170 columns
Presolve time: 0.00s
Presolved: 179 rows, 71 columns, 595 nonzeros
Variable types: 1 continuous, 70 integer (70 binary)

Root relaxation: objective 3.779552e-01, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.37796    0    6    0.83316    0.37796  54.6%     -    0s
H    0     0                       0.7997961    0.37796  5

In [147]:
mylist = []
obj = m.getObjective().getValue()
for v in x.values():
    mylist.append(v.x)
print('Integer Solution:')
print(mylist)
print('Objective Value:')
print(obj)

Integer Solution:
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Objective Value
0.6143520166745567


In [156]:
print('extra')

extra
